In [7]:
from sklearn.model_selection import train_test_split

In [2]:
import pandas as pd
df = pd.read_csv('final_dataset.csv')
pd.set_option('display.max_columns', None)
df.head()

,Unnamed: 0,review_date,is_spoiler,rating_x,movie_id_y,duration,rating_y,release_date,review_without_stopwords,description_without_stopwords,review_summary_without_stopwords,plot_summary_without_stopwords,Action,Adventure,Animation,Biography,Comedy,Crime,Drama,Family,Fantasy,Film-Noir,History,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,0,2006,True,10,tt0111161,142,9.3,1994,"In Oscar year, Shawshank Redemption (written d...","In 1947, Andy Dufresne (Tim Robbins), banker M...",A classic piece unforgettable film-making.,Chronicles experiences formerly successful ban...,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,2000,True,10,tt0111161,142,9.3,1994,The Shawshank Redemption without doubt one bri...,"In 1947, Andy Dufresne (Tim Robbins), banker M...",Simply amazing. The best film 90's.,Chronicles experiences formerly successful ban...,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,2001,True,8,tt0111161,142,9.3,1994,"I believe film best story ever told film, I'm ...","In 1947, Andy Dufresne (Tim Robbins), banker M...",The best story ever told film,Chronicles experiences formerly successful ban...,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,2002,True,10,tt0111161,142,9.3,1994,"**Yes, SPOILERS here**This film emotional impa...","In 1947, Andy Dufresne (Tim Robbins), banker M...",Busy dying busy living?,Chronicles experiences formerly successful ban...,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,2004,True,8,tt0111161,142,9.3,1994,At heart extraordinary movie brilliant indelib...,"In 1947, Andy Dufresne (Tim Robbins), banker M...","Great story, wondrously told acted",Chronicles experiences formerly successful ban...,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [3]:
import re


def clean_text(text):
    """
    Applies some pre-processing on the given text.

    Steps :
    - Removing HTML tags
    - Removing punctuation
    - Lowering text
    """
    
    # remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # remove the characters [\], ['] and ["]
    text = re.sub(r"\\", "", text)    
    text = re.sub(r"\'", "", text)    
    text = re.sub(r"\"", "", text)    
    
    # convert text to lowercase
    text = text.strip().lower()
    
    # replace punctuation characters with spaces
    filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    translate_dict = dict((c, " ") for c in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)

    return text


In [5]:
df["review_summary_without_stopwords"] = df["review_summary_without_stopwords"].astype(str)

In [8]:
X = df["review_summary_without_stopwords"].apply(lambda x: clean_text(x))
y = df['is_spoiler']
y = y.apply(lambda x: 1 if x else 0)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [10]:
y_train

64659     0
234573    0
24086     1
483424    0
342693    0
         ..
110268    0
259178    0
365838    1
131932    0
121958    1
Name: is_spoiler, Length: 430429, dtype: int64

In [14]:
df_train = pd.concat([X_train,y_train],axis=1)
df_train = df_train.rename(columns={"review_summary_without_stopwords": "text", "is_spoiler": "labels"})
df_train

,text,labels
64659,a superb masterpiece,0
234573,i loved movie,0
24086,one best,1
483424,makes first one look like citizen kane,0
342693,plot hole,0
...,...,...
110268,visual masterpiece one best sequels ever made,0
259178,dramatic muddled thriller,0
365838,not i expected,1
131932,think seen all,0


In [15]:
df_test = pd.concat([X_test,y_test],axis=1)
df_test = df_test.rename(columns={"review_summary_without_stopwords": "text", "is_spoiler": "labels"})
df_test

,text,labels
389140,the list,0
235482,well escalated quickly,1
294615,a cinematic triumph,0
336345,better reviews said,0
378675,great movie another peter jackson masterpie...,0
...,...,...
126785,better first 2 things bothered me,1
126711,witty emotionally satisfying topped impressi...,0
148034,a fun filled adventure,0
103883,applause hong kong bred director whose film li...,0


In [2]:
df['labels'] = list(zip(df.Action.tolist(), df.Adventure.tolist(), df.Animation.tolist(), df.Biography.tolist(),  df.Comedy.tolist(), df.Crime.tolist(), df.Drama.tolist(), df.Family.tolist(), df.Fantasy.tolist(), df.History.tolist(), df.Horror.tolist(), df.Music.tolist(), df.Musical.tolist(), df.Mystery.tolist(), df.Romance.tolist(), df.Sport.tolist(), df.Thriller.tolist(), df.War.tolist(), df.Western.tolist()))
df['review'] = df['review_without_stopwords'].apply(lambda x: x.replace('\n', ' '))
#df['desc'] = df['description_without_stopwords'].apply(lambda x: x.replace('\n', ' '))
#df['reviewSummary'] = df['review_summary_without_stopwords'].apply(lambda x: x.replace('\n', ' '))
#df['plot'] = df['plot_without_stopwords'].apply(lambda x: x.replace('\n', ' '))

df.head()

,Unnamed: 0,review_date,is_spoiler,rating_x,movie_id_y,duration,rating_y,release_date,review_without_stopwords,description_without_stopwords,review_summary_without_stopwords,plot_summary_without_stopwords,Action,Adventure,Animation,Biography,Comedy,Crime,Drama,Family,Fantasy,Film-Noir,History,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western,labels,review
0,0,2006,True,10,tt0111161,142,9.3,1994,"In Oscar year, Shawshank Redemption (written d...","In 1947, Andy Dufresne (Tim Robbins), banker M...",A classic piece unforgettable film-making.,Chronicles experiences formerly successful ban...,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","In Oscar year, Shawshank Redemption (written d..."
1,1,2000,True,10,tt0111161,142,9.3,1994,The Shawshank Redemption without doubt one bri...,"In 1947, Andy Dufresne (Tim Robbins), banker M...",Simply amazing. The best film 90's.,Chronicles experiences formerly successful ban...,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",The Shawshank Redemption without doubt one bri...
2,2,2001,True,8,tt0111161,142,9.3,1994,"I believe film best story ever told film, I'm ...","In 1947, Andy Dufresne (Tim Robbins), banker M...",The best story ever told film,Chronicles experiences formerly successful ban...,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","I believe film best story ever told film, I'm ..."
3,3,2002,True,10,tt0111161,142,9.3,1994,"**Yes, SPOILERS here**This film emotional impa...","In 1947, Andy Dufresne (Tim Robbins), banker M...",Busy dying busy living?,Chronicles experiences formerly successful ban...,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","**Yes, SPOILERS here**This film emotional impa..."
4,4,2004,True,8,tt0111161,142,9.3,1994,At heart extraordinary movie brilliant indelib...,"In 1947, Andy Dufresne (Tim Robbins), banker M...","Great story, wondrously told acted",Chronicles experiences formerly successful ban...,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",At heart extraordinary movie brilliant indelib...


In [3]:
from sklearn.model_selection import train_test_split


train_df, eval_df = train_test_split(df, test_size=0.2)

In [17]:
from simpletransformers.classification import ClassificationModel


model = ClassificationModel('roberta', 'roberta-base', num_labels=1, args={'train_batch_size':2, 'gradient_accumulation_steps':16, 'learning_rate': 3e-5, 'num_train_epochs': 3, 'max_seq_length': 512},use_cuda=False)


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

In [ ]:
model.train_model(df_train)